# Пикник в Москве

## Intro

Что делают программисты? Правильно, они делают понятные вещи странными способами.

Смотрят в окно, например: https://www.isitdarkoutside.com

Попробуем узнать, где в Москве лучше всего устроить летний пикник.

Но вместо того, чтобы спрашивать друзей и гулять по окрестностям, попробуем сделать это аналитически.

In [ ]:
import json
import numpy as np
import requests

## Получим данные

Документация: https://apidata.mos.ru/Docs

Получить токен: https://apidata.mos.ru/Account/Login

In [ ]:
# TODO получить токен с mos.ru, положить его в правильный файл и скачать датасет 
with open('./token.txt', 'r') as token:
    apiKey = token.read()

req_template = 'https://apidata.mos.ru/v1/{}/?api_key={}'
dataset_path = 'datasets/912/rows'  # это датасет про все места для пикников в Москве

response = requests.get(req_template.format(dataset_path, apiKey))

In [ ]:
data = response.json()

In [ ]:
response, len(data)

In [ ]:
data[0]

Отлично, у нас есть данные про 156 потенциальных мест для пикника в таком формате

Сохраним это куда-нибудь.

In [ ]:
with open('picnic.json', 'w') as f:
    json.dump(data, f)

## Изучим данные

In [ ]:
picnic_data = [row['Cells'] for row in data]

print('Всего мест', len(picnic_data))

In [ ]:
picnic_data[0]

Итак, у нас есть данные про (навскидку):
* координаты места
* его имя, адрес и всякие контакты (веб-сайт, район москвы, ...)
* его свойства (наличие wifi, туалетов, ...)

Важно понимать, что эти данные есть не всегда. 

Например, для Алтуфьевского заказника нет никакой информации про его работу зимой.

## Проверим полноту данных

Перед тем, как делать выводы из имеющихся данных, полезно посмотреть на их полноту.

Вдруг мы анализируем информацию, которой у нас вообще нет?

In [ ]:
for field in picnic_data[0].keys():
    picnics_without_field = sum([1 for picnic in picnic_data if picnic[field] is None])
    if picnics_without_field > 0:
        print('Поле {} не заполнено у {} мест'.format(field, picnics_without_field))

Поскольку у нас всего 156 мест, то можно понимать, какими полями интересоваться в принципе бесполезно.

# Преобразуем данные

```'HasWifi': 'нет'```

Не очень удобный формат для работы.
Строки ```"да"/"нет"``` лучше представить как логические (булевы) переменные (```True/False```).

Так:

1) данные будут занимать меньше места, т.к. строка ```"нет"``` занимает больше памяти, чем ```False```;

2) с данными будет удобнее работать.

Для этого:
* определите все поля, которые можно представить булевыми переменными;
* перезапишите значения этих полей.

In [ ]:
# TODO бинаризовать данные


In [ ]:
if not picnic_data[0]['HasWifi']:  # куда красивее, чем picnic_data[0]['HasWifi'] == 'нет'
    print('В {} нет Wifi... :('.format(picnic_data[0]['ObjectName']))

Ещё можно бы удалить поля, которые нам не очень интересны (любая информация про зиму не нужна в контексте летнего пикника).

Но сейчас не будем.

С булевыми полями всё ясно, а какие варианты бывают у "Paid"?

In [ ]:
set([picnic['Paid'] for picnic in picnic_data])

Хм, не очень интересно..

Кстати, та же история с Lighting -- нигде нет дополнительного освещения. :(

In [ ]:
set([(picnic['WebSite'], picnic['Email']) for picnic in picnic_data])

Т.к. поле ```WebSite``` было заполнено у всех мест, то ясно, что разнообразия в веб-сайтах нет.

# Займёмся делом
## То есть порисуем красивые картинки

Самая простая инфографика -- просто диаграммы со статистиками про разные свойства мест для пикника.

Посмотрим, насколько Москва в принципе обустроена.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline  # это нужно, чтобы графики рисовались прямо в ноутбуке
plt.rcParams.update({'font.size': 20})

In [ ]:
properties = (
    'HasCashMachine',
    'HasEatery',
    'HasFirstAidPost',
    'HasMusic',
    'HasToilet',
    'HasWifi'
)

In [ ]:
fig, axes= plt.subplots(2, 3, figsize=(15,10))
total = len(picnic_data)

for i, ax in enumerate(axes.flatten()):
    picnic_property = properties[i]
    has_property = sum(1 for picnic in picnic_data if picnic[picnic_property])
    ax.pie([has_property, total - has_property], labels=['да', 'нет'], radius=1, pctdistance=0.9)
    ax.set_title(picnic_property[3:])

plt.show()

По районам:

In [ ]:
fig1, ax1 = plt.subplots(figsize=(15, 15))

labels = set([picnic['District'] for picnic in picnic_data])
sizes = [sum([1 for picnic in picnic_data if picnic['District'] == label]) for label in labels]

# отсортируем по количеству мест для пикника в районе
labels, sizes = zip(*sorted(zip(labels, sizes), key=lambda x: x[1], reverse=True))

ax1.pie(sizes, labels=labels, explode=[0.2] * len(labels), autopct='%1.1f%%')
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()


Посмотрим на координаты мест для пикника.

In [ ]:
# TODO: подставьте координаты своего дома. Узнать их можно, например, в Яндекс.Картах.
my_coords = # . . .

In [ ]:
def plot_picnic(data, where_am_i=my_coords):
    plt.rcParams.update({'font.size': 20})
    
    points = np.array([picnic['geoData']['coordinates'] for picnic in data])

    plt.figure(figsize=(10,7))
    plt.plot(points[:,0], points[:,1], 'o', markersize=7, color='gray')
    plt.plot(*where_am_i, 'v', markersize=10, color='black')
    plt.annotate('я тут!', xy=where_am_i, xytext=(-15, -20), textcoords='offset points')
    plt.grid(True)
    plt.xlabel('долгота')
    plt.ylabel('широта')
    plt.show()

In [ ]:
plot_picnic(picnic_data)

Как-то нам не повезло -- все парки довольно далеко.

Не очень ясно, какой парк ближе, да и всё серое. Давайте раскрасим эту картинку!

In [ ]:
import matplotlib.cm as cm

def plot_picnic_colored(data, where_am_i=my_coords):
    plt.rcParams.update({'font.size': 20})
    
    titles = np.array([picnic['Address'] for picnic in data])
    points = np.array([picnic['geoData']['coordinates'] for picnic in data])
    # получим расстояния
    distances = [np.linalg.norm(point - where_am_i) for point in points]
    # нормализуем их, чтобы было красивое распределение
    distances_norm = [(distance - min(distances)) / (max(distances) - min(distances)) for distance in distances]
    # и нормализованные цвета для них, чем краснее -- тем ближе
    colors = map(cm.autumn, distances_norm)
    
    plt.figure(figsize=(10,7))
    for point, color, title in zip(points, colors, titles):
        plt.plot(*point, 'o', markersize=7, color=color)
    plt.plot(*where_am_i, 'v', markersize=10, color='black')
    plt.annotate('я тут!', xy=where_am_i, xytext=(-15, -20), textcoords='offset points')
    plt.grid(True)
    plt.xlabel('долгота')
    plt.ylabel('широта')
    plt.show()

In [ ]:
plot_picnic_colored(picnic_data)

Давайте научимся фильтровать места для пикника по каким-то свойствам!

In [ ]:
import matplotlib.cm as cm

def plot_only_good_picnic(data, condition=lambda x: True, where_am_i=my_coords):
    plt.rcParams.update({'font.size': 20})
    
    data = [picnic for picnic in data if condition(picnic) == True]
    plot_picnic_colored(data, where_am_i)

In [ ]:
plot_only_good_picnic(picnic_data)

In [ ]:
def has_toilet(place):
    return place['HasToilet']

In [ ]:
plot_only_good_picnic(picnic_data, condition=has_toilet)

In [ ]:
def has_music_and_food(place):
    return place['HasMusic'] and place['HasEatery']

In [ ]:
plot_only_good_picnic(picnic_data, condition=has_music_and_food)

## Дополнительные идеи

* подписать на карте адрес/название у 3 ближайших парков
* составить другие условия (как has_music_and_food)
* . . . anything!